# Simple tool to read the demo JSON data model file

In [ ]:
import os
import json
import pandas as pd

### Load the JSON file

In [ ]:
cwd = os.getcwd()
file_path = os.path.relpath(r'..\demo_data\iea43_wra_data_model.json', cwd)

with open(file_path) as json_file:
    meta_data = json.load(json_file)

### Show all the measurement locations

In [ ]:
 meta_data

In [ ]:
# print a table of the meas_locs parameters.
meas_locs = []
for meas_loc in meta_data['measurement_location']:
    meas_locs.append({
        'Name': meas_loc['name'],
        'Latitude': meas_loc['latitude_ddeg'],
        'Longitude': meas_loc['longitude_ddeg'],
        'Type': meas_loc['measurement_station_type_id']
    })
meas_locs_df = pd.DataFrame(meas_locs)
meas_locs_df.set_index('Name', inplace=True)
meas_locs_df

In [ ]:
#def get_meas_sensor_configs(meas_loc_uuid, remove_duplicates=True, add_sensor_config_attr_to_ignore=None):
# Keys ignored when assessing if there are any configuration duplicates
remove_duplicates = True
ignore_sensor_config_attr = ['id', 'notes', 'date_from', 'date_to', 'logger_main_config_uuid']

# meas_points = bw.load.load._LoadBWPlatform.get_meas_points(meas_loc_uuid)
# meas_points = _sort_meas_points(meas_points)
meas_points = meas_loc['measurement_point']
# for meas_point in meas_points:
#     meas_point['meas_point_notes'] = meas_point.pop('notes')

meas_sensor_configs = []
for meas_point in meas_points:
#     sen_configs = bw.load.load._LoadBWPlatform.get_sensor_configs(meas_point['id'])
    sen_configs = meas_point['sensor_config']
    sen_configs = sorted(sen_configs, key=lambda i: i['date_from'])
    #sen_configs = [_flatten_sensor_config_dict(sen_config) for sen_config in sen_configs]
    # merge similar sensors configs if the config is the same (except 'ignore_sensor_config_attr' keys)
    if remove_duplicates:
        for s in range(1, len(sen_configs)):
            if {k: v for k, v in sorted(sen_configs[s - 1].items()) if k not in ignore_sensor_config_attr} == \
                    {k: v for k, v in sorted(sen_configs[s].items()) if k not in ignore_sensor_config_attr}:
                for k in ignore_sensor_config_attr:
                    if (k != 'date_to') & (k != 'date_from') & (k in sen_configs[s].keys()):
                        if type(sen_configs[s-1][k]) is not list:
                            sen_configs[s][k] = [sen_configs[s - 1][k], sen_configs[s][k]]
                        else:
                            sen_configs[s-1][k].append(sen_configs[s][k])
                            sen_configs[s][k] = sen_configs[s - 1][k]

                sen_configs[s]['date_from'] = sen_configs[s - 1]['date_from']
                sen_configs[s - 1] = {}

        sen_configs = list(filter(None, sen_configs))

    for sen_config in sen_configs:
        sen_config['date_from'] = sen_config['date_from'].split('+')[0]
        sen_config['date_to'] = sen_config['date_to'].split('+')[0] if sen_config['date_to'] is not None else None
        #sen_config['sensor_config_id'] = sen_config['id']
        sen_config.update(meas_point)
        #del sen_config['id']
        if sen_config.get('desired_adj'):
            del sen_config['desired_adj']
        meas_sensor_configs.append(sen_config)

#return meas_sensor_configs

In [ ]:
meas_sensor_configs[0].keys()

In [ ]:
meas_sensor_configs[0]['sensor_config']

### Show all the measurement points with all their changes

In [ ]:
# print a table of the meas_points.


In [ ]:
# print a table of wind speed information only including all the config changes